## 本程式碼為整合了物體偵測模型(YOLOv5)、避障模型(Alexnet)，及控制戰車進行物體追蹤、噴灑藥劑與回傳任務訊息給使用者的程式碼
* 利用物體偵測所取得的BBox座標資訊計算BBox中心點，並計算該點與畫面正中心點座標的x軸距離、y軸距離、兩點距離來調整戰車角度、戰車速度並鎖定噴灑
* 當戰車未偵測到物體時進行遊走避障模式
* 在任務進行中會透過linebot以及imgur套件將任務執行相關文字訊息或照片傳送給使用者

## Load our object detection(YOLOv5) and collision detection(Alexnet) model

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('./models/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x


# 載入自行訓練的 YOLOv5 模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/yolov5s62best.pt', force_reload=True)

# 設定 IoU 門檻值
model.iou = 0.3

# 設定信心門檻值
model.conf = 0.5


def preprocess_yolo(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (224, 224))
    return x

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-9-29 Python-3.6.9 torch-1.8.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)



WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## Create a camera instance

In [2]:
from jetbot import Camera

camera = Camera.instance(width=224, height=224)

## Initialize Jetbot so we can control the motors

In [6]:
from jetbot import Robot

robot = Robot()

## Spray function

In [4]:
import Jetson.GPIO as GPIO
import time


output_pin = 21

def spray():
    start_spray_time = time.time()
    
# 設置為BCM模式
    GPIO.setmode(GPIO.BCM) 
    
# 將腳位設定為輸出，並預設為輸出高電位

    GPIO.setup(output_pin, GPIO.OUT, initial=GPIO.HIGH)

    print("Starting demo now! Press CTRL+C to exit")
    curr_value = GPIO.HIGH
    try:
        while True:
            end_spray_time = time.time()
            spray_interval = end_spray_time - start_spray_time 
            if 0 < spray_interval <= 3:
            # 每隔2.5秒鐘切換
                time.sleep(2.5)
                print("Outputting {} to pin {}".format(curr_value, output_pin))
                GPIO.output(output_pin, curr_value)
                curr_value ^= GPIO.HIGH
            else:
                break
    finally:
        GPIO.cleanup()

# if __name__ == '__main__':
#     main()

## Snap function

In [9]:
import os
from uuid import uuid1

def save_snapshot_found():
    # image_path = os.path.join('/workspace/jetbot/notebooks/object_following/', str(uuid1()) + '.jpg')
    image_path = os.path.join('/workspace/jetbot/notebooks/object_following/', 'cockroach_found' + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        
def save_snapshot_end():
    # image_path = os.path.join('/workspace/jetbot/notebooks/object_following/', str(uuid1()) + '.jpg')
    image_path_end = os.path.join('/workspace/jetbot/notebooks/object_following/', 'mission_end' + '.jpg')
    with open(image_path_end, 'wb') as f:
        f.write(image_widget.value)
        
        
def save_snapshot_kill():
    # image_path = os.path.join('/workspace/jetbot/notebooks/object_following/', str(uuid1()) + '.jpg')
    image_path_end = os.path.join('/workspace/jetbot/notebooks/object_following/', 'cockroach_kill' + '.jpg')
    with open(image_path_end, 'wb') as f:
        f.write(image_widget.value)

## Display all the control widgets and connect the network execution function to the camera updates

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import bgr8_to_jpeg
import time
from linebot import LineBotApi
from linebot.models import TextSendMessage, ImageSendMessage
import pyimgur
import pandas as pd

start_time = time.time()


# LineBot upload
# 必須放上自己的Channel Access Token
line_bot_api = LineBotApi('zYXfWsIEAgAiqGoU62iLIdiPDsIVsz5t1W0lr3urkwyemRmrfTZBCKauljVvoYqKFX71W08VEzNuIceqtoqWRBIiAefZAo8fKXJV9HgpxuCv+DXUIbVc5v8RwsMbRqb9J014bZrgo+e3TYZoPD3Y4AdB04t89/1O/w1cDnyilFU=')
# 請填入您的ID
yourID = 'U61a52960fc6fb48ae57da4f63bf06497'

CLIENT_ID = "e0e5447c40d7cef"
image_path = '/workspace/jetbot/notebooks/object_following/cockroach_found.jpg'
image_path_kill = '/workspace/jetbot/notebooks/object_following/cockroach_kill.jpg'
image_path_end = '/workspace/jetbot/notebooks/object_following/mission_end.jpg'


# Widget set up
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=224, height=224)

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget])
]))



frame_width = int(image_widget.width)
frame_height = int(image_widget.height)

frame_central_x = 112
frame_central_y = 112

found = 0

spray_times = 0

while True:
    end_time = time.time()
    interval_time = end_time - start_time
    
    if interval_time < 90:
        image = camera.value

        # execute collision model to determine if blocked
        collision_output = collision_model(preprocess(image)).detach().cpu()
        prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
        blocked_widget.value = prob_blocked

        # compute all detected objects
        detections = model(preprocess_yolo(image), size=224)
        
        # draw all detections on image  # 如果畫面中有蟑螂
        if not detections.pandas().xyxy[0].empty:
            if found == 0:
                line_bot_api.push_message(yourID, TextSendMessage(text='主人！在您家發現蟑螂了！'))
                found = 1
                
            length_detections = len(detections.pandas().xyxy[0].index)  # calculation of how many detections in a frame
            
            for i in range(length_detections):
                xmin = int(detections.pandas().xyxy[0].loc[i]['xmin'])
                xmax = int(detections.pandas().xyxy[0].loc[i]['xmax'])
                ymin = int(detections.pandas().xyxy[0].loc[i]['ymin'])
                ymax = int(detections.pandas().xyxy[0].loc[i]['ymax'])
                w = xmax - xmin
                h = ymax - ymin
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)  # draw the BBoxs

            
            # select detections which class name is cockroach because we have two classes indicate live or dead
            matching_df = detections.pandas().xyxy[0]
            filter = (matching_df['name'] == 'cockroach')
            matching_df = matching_df[filter]

            # select detection that has the highest confidence score
            matching_detections = matching_df.loc[0]

            # draw the target circle
            match_xmin = matching_detections['xmin']
            match_xmax = matching_detections['xmax']
            match_ymin = matching_detections['ymin']
            match_ymax = matching_detections['ymax']

            match_w = match_xmax - match_xmin
            central_x = int(match_xmin + (match_w/2))
            match_h = match_ymax - match_ymin
            central_y = int(match_ymin + (h/2))
            
            x_distance = central_x - frame_central_x
            y_distance = central_y - frame_central_y

            cv2.circle(image, (central_x, central_y), 10, (255, 255, 0), 2)

            # calculate the distance from frame_center to BBox center (target)
            distance = np.sqrt((frame_central_x - central_x)**2 + (frame_central_y - central_y)**2)
            cv2.line(image, (frame_central_x, frame_central_y), (central_x, central_y), (0, 255, 0), 2)
            
            
            # BBOX x = 88~120 => go forward
            # spray distance = 14~23 => stop and spray ; central_y = 103~139
            
            if x_distance > 0:  # 蟑螂在右前方
                if central_x >= 120:
                    if distance < 14 or distance > 31: 
                        print("蟑螂在右前方")
                        robot.right(speed=0.2)
                        x_time = (x_distance/7.1)
                        x_time *= 0.1
                        time.sleep(x_time)
                        robot.stop()
                        time.sleep(0.01)
                        print("已經往右跑")
                        
                    elif 14 <= distance <= 31 and 102 <= central_y:
                        print("停止")
                        robot.stop()
                        line_bot_api.push_message(yourID, TextSendMessage(text='撲殺ing...'))
                        spray()
#                         save_snapshot_found()
#                         time.sleep(0.5)
#                         im = pyimgur.Imgur(CLIENT_ID)
#                         uploaded_image = im.upload_image(image_path)
#                         link = uploaded_image.link
#                         line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link, preview_image_url=link))
                        spray_times += 1
                        # if matching_df.empty or None:
                        if spray_times == 2:
                            print("滅蟑完成！")
                            line_bot_api.push_message(yourID, TextSendMessage(text='主人，已撲殺！'))
                            time.sleep(2)
                            save_snapshot_kill()
                            time.sleep(0.5)
                            im_kill = pyimgur.Imgur(CLIENT_ID)
                            uploaded_image_kill = im_kill.upload_image(image_path_kill)
                            link_kill = uploaded_image_kill.link
                            line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_kill, preview_image_url=link_kill))
                            time.sleep(1)
                            robot.left(0.3)
                            time.sleep(1)
                            spray_times = 0
                            
                    elif distance >= 31 and 102 <= central_y:
                        print('太靠近蟑螂！')
                        robot.backward(0.3)
                        time.sleep(1)

                elif 112 <= central_x < 120:
                    if distance < 14 or 31 < distance < 40:
                        print("往前走_蟑螂在微右前方")
                        robot.set_motors(0.4, 0.4)
                        print("已經往前走_右前")
                        
                    elif 40 <= distance <= 55:
                        print("加速往前走_蟑螂在微右前方")
                        robot.set_motors(0.6, 0.6)
                        print("已經加速往前走_右前")
                        
                    elif 14 <= distance <= 31 and 102 <= central_y:
                        print("停止")
                        robot.stop()
                        line_bot_api.push_message(yourID, TextSendMessage(text='撲殺ing...'))
                        spray()
#                         save_snapshot_found()
#                         time.sleep(0.5)
#                         im = pyimgur.Imgur(CLIENT_ID)
#                         uploaded_image = im.upload_image(image_path)
#                         link = uploaded_image.link
#                         line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link, preview_image_url=link))
                        spray_times += 1
                        # if matching_df.empty or None:
                        if spray_times == 2:
                            print("滅蟑完成！")
                            line_bot_api.push_message(yourID, TextSendMessage(text='主人，已撲殺！'))
                            time.sleep(2)
                            save_snapshot_kill()
                            time.sleep(0.5)
                            im_kill = pyimgur.Imgur(CLIENT_ID)
                            uploaded_image_kill = im_kill.upload_image(image_path_kill)
                            link_kill = uploaded_image_kill.link
                            line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_kill, preview_image_url=link_kill))
                            time.sleep(1)
                            robot.left(0.3)
                            time.sleep(1)
                            spray_times = 0
                            
                    elif distance >= 31 and 102 <= central_y:
                        print('太靠近蟑螂！')
                        robot.backward(0.3)
                        time.sleep(1)
                        
            elif x_distance < 0:  # 蟑螂在左前方
                if 88 <= central_x <= 112:
                    if distance < 14 or 31 < distance < 45:
                        print("往前走_蟑螂在微左前方")
                        robot.set_motors(0.4, 0.4)
                        print("已經往前走_左前")
                        
                    elif 45 <= distance <= 60:
                        print("加速往前走_蟑螂在微左前方")
                        robot.set_motors(0.6, 0.6)
                        print("已經加速往前走_左前")
                        
                    elif 14 <= distance <= 31 and 102 <= central_y:
                        print("停止")
                        robot.stop()
                        line_bot_api.push_message(yourID, TextSendMessage(text='撲殺ing...'))
                        spray()
#                         save_snapshot_found()
#                         time.sleep(0.5)
#                         im = pyimgur.Imgur(CLIENT_ID)
#                         uploaded_image = im.upload_image(image_path)
#                         link = uploaded_image.link
#                         line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link, preview_image_url=link))
                        spray_times += 1
                        # if matching_df.empty or None:
                        if spray_times == 2:
                            print("滅蟑完成！")
                            line_bot_api.push_message(yourID, TextSendMessage(text='主人，已撲殺！'))
                            time.sleep(2)
                            save_snapshot_kill()
                            time.sleep(0.5)
                            im_kill = pyimgur.Imgur(CLIENT_ID)
                            uploaded_image_kill = im_kill.upload_image(image_path_kill)
                            link_kill = uploaded_image_kill.link
                            line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_kill, preview_image_url=link_kill))
                            time.sleep(1)
                            robot.left(0.3)
                            time.sleep(1)
                            spray_times = 0
                    
                    elif distance >= 31 and 102 <= central_y:
                        print('太靠近蟑螂！')
                        robot.backward(0.3)
                        time.sleep(1)
                        
                        
                elif central_x < 88:
                    if distance < 14 or distance > 31:
                        print("蟑螂在左前方")
                        robot.left(speed=0.2)
                        x_time = (x_distance/6.3)
                        x_time *= -0.1
                        time.sleep(x_time)
                        robot.stop()
                        time.sleep(0.01)
                        print("已經往左跑了")
                        
                    elif 14 <= distance <= 31 and 102 <= central_y:
                        print("停止")
                        robot.stop()
                        line_bot_api.push_message(yourID, TextSendMessage(text='撲殺ing...'))
                        spray()
#                         save_snapshot_found()
#                         time.sleep(0.5)
#                         im = pyimgur.Imgur(CLIENT_ID)
#                         uploaded_image = im.upload_image(image_path)
#                         link = uploaded_image.link
#                         line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link, preview_image_url=link))
                        spray_times += 1
                        # if matching_df.empty or None:
                        if spray_times == 2:
                            print("滅蟑完成！")
                            line_bot_api.push_message(yourID, TextSendMessage(text='主人，已撲殺！'))
                            time.sleep(2)
                            save_snapshot_kill()
                            time.sleep(0.5)
                            im_kill = pyimgur.Imgur(CLIENT_ID)
                            uploaded_image_kill = im_kill.upload_image(image_path_kill)
                            link_kill = uploaded_image_kill.link
                            line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_kill, preview_image_url=link_kill))
                            time.sleep(1)
                            robot.left(0.3)
                            time.sleep(1)
                            spray_times = 0
                            
                    elif distance >= 31 and 102 <= central_y:
                        print('太靠近蟑螂！')
                        robot.backward(0.3)
                        time.sleep(1)
                        
                        
            elif x_distance == 0:
                if distance < 14 or 31 < distance < 45:
                    print("蟑螂在正中間")
                    robot.set_motors(0.4, 0.4)
                    robot.stop()
                    time.sleep(0.001)
                    print("已經跑完了")
                    
                elif 45 <= distance <= 60:
                    print("加速往前走_蟑螂在正中間")
                    robot.set_motors(0.6, 0.6)
                    print("已經加速往前走")
                    
                elif 14 <= distance <= 31 and 102 <= central_y:
                    print("停止")
                    robot.stop()
                    line_bot_api.push_message(yourID, TextSendMessage(text='撲殺ing...'))
                    spray()
#                     save_snapshot_found()
#                     time.sleep(0.5)
#                     im = pyimgur.Imgur(CLIENT_ID)
#                     uploaded_image = im.upload_image(image_path)
#                     link = uploaded_image.link
#                     line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link, preview_image_url=link))
                    spray_times += 1
                    # if matching_df.empty or None:
                    if spray_times == 2:
                        print("滅蟑完成！")
                        line_bot_api.push_message(yourID, TextSendMessage(text='主人，已撲殺！'))
                        time.sleep(2)
                        save_snapshot_kill()
                        time.sleep(0.5)
                        im_kill = pyimgur.Imgur(CLIENT_ID)
                        uploaded_image_kill = im_kill.upload_image(image_path_kill)
                        link_kill = uploaded_image_kill.link
                        line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_kill, preview_image_url=link_kill))
                        time.sleep(1)
                        robot.left(0.3)
                        time.sleep(1)
                        spray_times = 0
                        
                elif distance >= 31 and 102 <= central_y:
                        print('太靠近蟑螂！')
                        robot.backward(0.3)
                        time.sleep(1)
                    
        # otherwise go forward if no target detected  # 若畫面中沒有蟑螂
        if detections.pandas().xyxy[0].empty:
            found = 0
            robot.set_motors(0.20, 0.20)

            # turn right if blocked
            if prob_blocked > 0.01:
                robot.right(0.5)
                image_widget.value = bgr8_to_jpeg(image)

        else:
            found = 0
            robot.set_motors(0.20, 0.20)
            if prob_blocked > 0.01:
                robot.left(0.5)
                image_widget.value = bgr8_to_jpeg(image)
    else:
        robot.stop()
        save_snapshot_end()
        time.sleep(1)
        im_end = pyimgur.Imgur(CLIENT_ID)
        uploaded_image_end = im_end.upload_image(image_path_end)
        link_end = uploaded_image_end.link
        line_bot_api.push_message(yourID, ImageSendMessage(original_content_url=link_end, preview_image_url=link_end))
        line_bot_api.push_message(yourID, TextSendMessage(text='守護家園任務結束！'))
        print('守護家園任務結束！')
        break
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    

加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
往前走_蟑螂在微左前方
已經往前走_左前
蟑螂在正中間
已經跑完了
往前走_蟑螂在微左前方
已經往前走_左前
蟑螂在右前方
已經往右跑
停止
Starting demo now! Press CTRL+C to exit
Outputting 1 to pin 21
Outputting 0 to pin 21
蟑螂在右前方
已經往右跑
太靠近蟑螂！
停止
Starting demo now! Press CTRL+C to exit
Outputting 1 to pin 21
Outputting 0 to pin 21
滅蟑完成！
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
加速往前走_蟑螂在微左前方
已經加速往前走_左前
往前走_蟑螂在微左前方
已經往前走_左前
往前走_蟑螂在微左前方
已經往前走_左前
往前走_蟑螂在微左前方
已經往前走_左前
往前走_蟑螂在微左前方
已經往前走_左前
往前走_蟑螂在微左前方
已經往前走_左前
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
停止
Starting demo now! Press CTRL+C to exit
Outputting 1 to pin 21
Outputting 0 to pin 21
往前走_蟑螂在微左前方
已經往前走_左前
蟑螂在左前方
已經往左跑了
蟑螂在右前方
已經往右跑
蟑螂在右前方
已經往右跑
蟑螂在右前方
已經往右跑
蟑螂在右前方
已經往右跑
蟑螂在右前方
已經往右跑
蟑螂在右前方
已經往右跑
太靠近蟑螂！


## Turn off the robot and camera

In [8]:
robot.stop()
camera.stop()